<a href="https://colab.research.google.com/github/Q815S/NE/blob/main/NE5_Gemini_Voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U google-generativeai
!pip install SpeechRecognition pydub

import google.generativeai as genai
import speech_recognition as sr
from pydub import AudioSegment
import pathlib,textwrap, os, requests,re


from IPython.display import display
from IPython.display import Markdown
from google.colab import userdata
from google.colab import files

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-pro-exp-0801')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 28.9 MB/s eta 0:00:00


In [ ]:
def recognize_speech(file_path):
  try:
    sound = AudioSegment.from_file(file_path, format='m4a')
    sound.export('audio.wav', format='wav')

    r = sr.Recognizer()
    with sr.AudioFile('audio.wav') as source:
      audio = r.record(source)
    VR = r.recognize_google(audio, language='ko')
    return VR

  except sr.UnknownValueError:
    print("음성을 인식할 수 없습니다.")
    return None

  except sr.RequestError as e:
    print(f"Could not request results from Google Speech Recognition service; {e}")
    return None

  finally:
    os.remove('audio.wav')

print('듣고 있어요.')

while True:
  myfiles = files.upload()
  VR = recognize_speech('audio.m4a')
  if VR:
    response = model.generate_content(VR)
    markdown_text = to_markdown(response.text)
    display(markdown_text)
    os.remove('audio.m4a')

  if VR == '고마워 잘가':
    print("안녕히 가세요!")
    break